<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия №3
<center>Автор материала: программист-исследователь Mail.Ru Group Юрий Кашницкий

# <center> Домашнее задание № 8
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на Stackoverflow

## План
    1. Введение
    2. Описание данных
    3. Предобработка данных
    4. Обучение и проверка моделей
    5. Заключение

### 1. Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.Ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

[Веб-форма](https://docs.google.com/forms/d/1VaxYXnmbpeP185qPk2_V_BzbeduVUVyTdLPQwSCxDGA/edit) для ответов.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в Wiki [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 70 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась около 12 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на час-полтора)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### 2. Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://drive.google.com/file/d/1ZU4J3KhJDrHVMj48fROFcTsTZKorPGlG/view) и распакуйте архив. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [1]:
!head -1 hw8_data/stackoverflow.10kk.tsv

head: cannot open `hw8_data/stackoverflow.10kk.tsv' for reading: No such file or directory


Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [7]:
%%time
!wc -l hw8_data/stackoverflow.10kk.tsv

10000000 hw8_data/stackoverflow.10kk.tsv
CPU times: user 108 ms, sys: 32 ms, total: 140 ms
Wall time: 5.16 s


Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

### 3. Предобработка данных

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлам на входе и на выходе
 - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
 - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
 - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 - *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
 - из текста вопроса надо выкинуть двоеточия и вертикальные палки, если они есть – в VW это спецсимволы

In [30]:
import os
from tqdm import tqdm
from time import time
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

Должно получиться вот такое число строк – 4389054. 10 Гб у меня обработались примерно за 2 минуты.

In [25]:
# !python3 hw8_preprocess.py hw8_data/stackoverflow.10kk.tsv hw8_data/stackoverflow.vw

['hw8_preprocess.py', 'hw8_data/stackoverflow.10kk.tsv', 'hw8_data/stackoverflow.vw']
[2018-04-09 23:32:27.208934] Labels: {'javascript': 1, 'java': 2, 'python': 3, 'ruby': 4, 'php': 5, 'c++': 6, 'c#': 7, 'go': 8, 'scala': 9, 'swift': 10}
[2018-04-09 23:33:01.102538] line #3398558: ' 	python django
'
[2018-04-09 23:33:05.576487] line #3890296: ' 	ruby-on-rails ruby bundle gemfile
'
[2018-04-09 23:33:09.769830] line #4350090: ' 	c++ arrays
'
[2018-04-09 23:33:24.681299] line #5987980: ' 	c# vb.net excel csv export-to-csv
'
[2018-04-09 23:34:10.397366] corrupted: 15, bad tags: 5610931, ok lines: 4389054


In [3]:
%%time
!wc -l hw8_data/stackoverflow.vw

4389054 hw8_data/stackoverflow.vw
CPU times: user 24 ms, sys: 8 ms, total: 32 ms
Wall time: 1.22 s


In [44]:
# !gzip hw8_data/stackoverflow.10kk.tsv

Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции - по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

In [41]:
!head -1463018 hw8_data/stackoverflow.vw > hw8_data/stackoverflow_train.vw

!head -2926036 hw8_data/stackoverflow.vw > hw8_data/tmp_stackoverflow_test.vw
!tail -1463018 hw8_data/tmp_stackoverflow_test.vw > hw8_data/stackoverflow_test.vw
!rm hw8_data/tmp_stackoverflow_test.vw

!tail -1463018 hw8_data/stackoverflow.vw > hw8_data/stackoverflow_valid.vw

In [42]:
!cut -c1 hw8_data/stackoverflow_test.vw > hw8_data/stackoverflow_test_labels.txt
!cut -c1 hw8_data/stackoverflow_valid.vw > hw8_data/stackoverflow_valid_labels.txt

In [43]:
!wc -l hw8_data/stackoverflow_train.vw
!wc -l hw8_data/stackoverflow_test.vw
!wc -l hw8_data/stackoverflow_valid.vw

!head -1 hw8_data/stackoverflow_train.vw
!head -1 hw8_data/stackoverflow_test.vw
!head -1 hw8_data/stackoverflow_valid.vw

!wc -l hw8_data/stackoverflow_test_labels.txt
!wc -l hw8_data/stackoverflow_valid_labels.txt

!head -1 hw8_data/stackoverflow_test_labels.txt
!head -1 hw8_data/stackoverflow_valid_labels.txt

1463018 hw8_data/stackoverflow_train.vw
1463018 hw8_data/stackoverflow_test.vw
1463018 hw8_data/stackoverflow_valid.vw
1 | i ve got some code in window scroll that checks if an element is visible then triggers another function however only the first section of code is firing both bits of code work in and of themselves if i swap their order whichever is on top fires correctly my code is as follows fn isonscreen function use strict var win window viewport top win scrolltop left win scrollleft bounds this offset viewport right viewport left + win width viewport bottom viewport top + win height bounds right bounds left + this outerwidth bounds bottom bounds top + this outerheight return viewport right lt bounds left viewport left gt bounds right viewport bottom lt bounds top viewport top gt bounds bottom window scroll function use strict var load_more_results ajax load_more_results isonscreen if load_more_results true loadmoreresults var load_more_staff ajax load_more_staff isonscreen if l

### 4. Обучение и проверка моделей

Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: `loss_function=hinge`, `bit_precision`=28 и `seed`=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

[HABR](https://habrahabr.ru/company/ods/blog/326418/)

In [28]:
# run models learning
params = {'passes': [1, 3, 5], 'ngram': [1, 2, 3]}
models = []
for passes in params['passes']:
    for ngram in params['ngram']:
        model_name = "passes_{}_ngram_{}".format(passes, ngram)
        print(
            '!vw --oaa 10 -d hw8_data/stackoverflow_train.vw -f hw8_data/models/{}_model.vw  \\\n'
            '    --loss_function hinge --bit_precision 28 --random_seed 17 --cache_file cache.vw \\\n'
            '    --passes {} --ngram {}\n'.format(model_name, passes, ngram)
        )
        models.append(model_name)

!vw --oaa 10 -d hw8_data/stackoverflow_train.vw -f hw8_data/models/passes_1_ngram_1_model.vw  \
    --loss_function hinge --bit_precision 28 --random_seed 17 --cache_file cache.vw \
    --passes 1 --ngram 1

!vw --oaa 10 -d hw8_data/stackoverflow_train.vw -f hw8_data/models/passes_1_ngram_2_model.vw  \
    --loss_function hinge --bit_precision 28 --random_seed 17 --cache_file cache.vw \
    --passes 1 --ngram 2

!vw --oaa 10 -d hw8_data/stackoverflow_train.vw -f hw8_data/models/passes_1_ngram_3_model.vw  \
    --loss_function hinge --bit_precision 28 --random_seed 17 --cache_file cache.vw \
    --passes 1 --ngram 3

!vw --oaa 10 -d hw8_data/stackoverflow_train.vw -f hw8_data/models/passes_3_ngram_1_model.vw  \
    --loss_function hinge --bit_precision 28 --random_seed 17 --cache_file cache.vw \
    --passes 3 --ngram 1

!vw --oaa 10 -d hw8_data/stackoverflow_train.vw -f hw8_data/models/passes_3_ngram_2_model.vw  \
    --loss_function hinge --bit_precision 28 --random_seed 17 --cache_fi

In [37]:
# run models applying
for model_name in models:
    print(
        '!vw -i hw8_data/models/{}_model.vw -t -d hw8_data/stackoverflow_valid.vw \\\n'
        '    -p hw8_data/models/{}_valid_answers.tsv\n'.format(model_name, model_name)
    )

!vw -i hw8_data/models/passes_1_ngram_1_model.vw -t -d hw8_data/stackoverflow_valid.vw \
    -p hw8_data/models/passes_1_ngram_1_valid_answers.tsv

!vw -i hw8_data/models/passes_1_ngram_2_model.vw -t -d hw8_data/stackoverflow_valid.vw \
    -p hw8_data/models/passes_1_ngram_2_valid_answers.tsv

!vw -i hw8_data/models/passes_1_ngram_3_model.vw -t -d hw8_data/stackoverflow_valid.vw \
    -p hw8_data/models/passes_1_ngram_3_valid_answers.tsv

!vw -i hw8_data/models/passes_3_ngram_1_model.vw -t -d hw8_data/stackoverflow_valid.vw \
    -p hw8_data/models/passes_3_ngram_1_valid_answers.tsv

!vw -i hw8_data/models/passes_3_ngram_2_model.vw -t -d hw8_data/stackoverflow_valid.vw \
    -p hw8_data/models/passes_3_ngram_2_valid_answers.tsv

!vw -i hw8_data/models/passes_3_ngram_3_model.vw -t -d hw8_data/stackoverflow_valid.vw \
    -p hw8_data/models/passes_3_ngram_3_valid_answers.tsv

!vw -i hw8_data/models/passes_5_ngram_1_model.vw -t -d hw8_data/stackoverflow_valid.vw \
    -p hw8_data/models/

In [38]:
# calculate metrics
valid_true_answers = pd.read_csv('hw8_data/stackoverflow_valid_labels.txt')

for model_name in models:
    model_answers = pd.read_csv('hw8_data/models/{}_valid_answers.tsv'.format(model_name))
    acc = accuracy_score(y_true=valid_true_answers, y_pred=model_answers)
    print(model_name, acc)

passes_1_ngram_1 0.896461900306
passes_1_ngram_2 0.911553317562
passes_1_ngram_3 0.909102218224
passes_3_ngram_1 0.897642337717
passes_3_ngram_2 0.90948977353
passes_3_ngram_3 0.906454265398
passes_5_ngram_1 0.897426345695
passes_5_ngram_2 0.910154154053
passes_5_ngram_3 0.907330536829


<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- **Биграммы и 1 проход по выборке**
- Униграммы и 1 проход по выборке

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

In [39]:
''' ВАШ КОД ЗДЕСЬ '''
!vw -i hw8_data/models/passes_1_ngram_2_model.vw -t -d hw8_data/stackoverflow_test.vw \
    -p hw8_data/models/passes_1_ngram_2_test_answers.tsv

Generating 2-grams for all namespaces.
only testing
predictions = hw8_data/models/passes_1_ngram_2_test_answers.tsv
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = hw8_data/stackoverflow_test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        2        7      354
0.500000 0.000000            2            2.0        7        7      146
0.250000 0.000000            4            4.0        5        5      516
0.125000 0.000000            8            8.0        7        7      286
0.125000 0.125000           16           16.0        6        6      716
0.062500 0.000000           32           32.0        2        2      798
0.062500 0.062500           64           64.0        5        5     2120
0.046875 0.031250          128          128.0        2        2      262
0.07

In [41]:
test_true_answers = pd.read_csv('hw8_data/stackoverflow_test_labels.txt')
model_answers = pd.read_csv('hw8_data/models/{}_test_answers.tsv'.format('passes_1_ngram_2'))
accuracy_score(y_true=test_true_answers, y_pred=model_answers)

0.91140157633164887

<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках? (здесь % – это процентный пункт, т.е., скажем, снижение с 50% до 40% – это на 10%, а не 20%).
- На тестовой ниже примерно на 2%
- На тестовой ниже примерно на 3%
- **Результаты почти одинаковы – отличаются меньше чем на 0.5%**

Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [42]:
!cat hw8_data/stackoverflow_train.vw hw8_data/stackoverflow_valid.vw > hw8_data/stackoverflow_train_valid.vw

In [48]:
''' ВАШ КОД ЗДЕСЬ '''
# NB! cache file

!vw --oaa 10 -d hw8_data/stackoverflow_train_valid.vw -f hw8_data/models/passes_1_ngram_2_ext_model.vw  \
    --bit_precision 28 --random_seed 17 --cache_file cache1.vw \
    --passes 1 --ngram 2
# --loss_function hinge

!vw -i hw8_data/models/passes_1_ngram_2_ext_model.vw -t -d hw8_data/stackoverflow_test.vw \
    -p hw8_data/models/passes_1_ngram_2_ext_test_answers.tsv

Generating 2-grams for all namespaces.
final_regressor = hw8_data/models/passes_1_ngram_2_ext_model.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = cache1.vw
Reading datafile = hw8_data/stackoverflow_train_valid.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      320
0.500000 1.000000            2            2.0        4        1      134
0.750000 1.000000            4            4.0        7        1      174
0.750000 0.750000            8            8.0        7        1      188
0.750000 0.750000           16           16.0        7        7      416
0.781250 0.812500           32           32.0        7        2      346
0.750000 0.718750           64           64.0        3        3      406
0.648438 0.546875          128          128.0        1        7   

In [50]:
test_true_answers = pd.read_csv('hw8_data/stackoverflow_test_labels.txt')
model_answers = pd.read_csv('hw8_data/models/{}_test_answers.tsv'.format('passes_1_ngram_2_ext'))
accuracy_score(y_true=test_true_answers, y_pred=model_answers)

0.91543843988142315

In [54]:
0.91543843988142315 - 0.911553317562

0.003885122319423173

<font color='red'> Вопрос 3.</font> На сколько процентных пунктов повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - **0.4%**
 - 0.8%
 - 1.2%

## Повторим то же самое для не hinge

In [56]:
# run models learning
models_sq = []
for passes in params['passes']:
    for ngram in params['ngram']:
        model_name = "passes_{}_ngram_{}_squared".format(passes, ngram)
        print(
            '!vw --oaa 10 -d hw8_data/stackoverflow_train.vw -f hw8_data/models/{}_model.vw  \\\n'
            '    --bit_precision 28 --random_seed 17 --cache_file cache.vw \\\n'
            '    --passes {} --ngram {}\n'.format(model_name, passes, ngram)
        )
        models_sq.append(model_name)

# run models applying
for model_name in models_sq:
    print(
        '!vw -i hw8_data/models/{}_model.vw -t -d hw8_data/stackoverflow_valid.vw \\\n'
        '    -p hw8_data/models/{}_valid_answers.tsv\n'.format(model_name, model_name)
    )

!vw --oaa 10 -d hw8_data/stackoverflow_train.vw -f hw8_data/models/passes_1_ngram_1_squared_model.vw  \
    --bit_precision 28 --random_seed 17 --cache_file cache.vw \
    --passes 1 --ngram 1

!vw --oaa 10 -d hw8_data/stackoverflow_train.vw -f hw8_data/models/passes_1_ngram_2_squared_model.vw  \
    --bit_precision 28 --random_seed 17 --cache_file cache.vw \
    --passes 1 --ngram 2

!vw --oaa 10 -d hw8_data/stackoverflow_train.vw -f hw8_data/models/passes_1_ngram_3_squared_model.vw  \
    --bit_precision 28 --random_seed 17 --cache_file cache.vw \
    --passes 1 --ngram 3

!vw --oaa 10 -d hw8_data/stackoverflow_train.vw -f hw8_data/models/passes_3_ngram_1_squared_model.vw  \
    --bit_precision 28 --random_seed 17 --cache_file cache.vw \
    --passes 3 --ngram 1

!vw --oaa 10 -d hw8_data/stackoverflow_train.vw -f hw8_data/models/passes_3_ngram_2_squared_model.vw  \
    --bit_precision 28 --random_seed 17 --cache_file cache.vw \
    --passes 3 --ngram 2

!vw --oaa 10 -d hw8_data/stack

In [57]:
# calculate metrics
valid_true_answers = pd.read_csv('hw8_data/stackoverflow_valid_labels.txt')

for model_name in models_sq:
    model_answers = pd.read_csv('hw8_data/models/{}_valid_answers.tsv'.format(model_name))
    acc = accuracy_score(y_true=valid_true_answers, y_pred=model_answers)
    print(model_name, acc)

passes_1_ngram_1_squared 0.894791379731
passes_1_ngram_2_squared 0.910434396866
passes_1_ngram_3_squared 0.907942286385
passes_3_ngram_1_squared 0.893807112289
passes_3_ngram_2_squared 0.906871895542
passes_3_ngram_3_squared 0.905299118192
passes_5_ngram_1_squared 0.893072329303
passes_5_ngram_2_squared 0.908384523215
passes_5_ngram_3_squared 0.905296384116


In [58]:
''' ВАШ КОД ЗДЕСЬ '''
# NB! cache file

!vw --oaa 10 -d hw8_data/stackoverflow_train_valid.vw -f hw8_data/models/passes_1_ngram_2_squared_ext_model.vw  \
    --bit_precision 28 --random_seed 17 --cache_file cache1.vw \
    --passes 1 --ngram 2
# --loss_function hinge

!vw -i hw8_data/models/passes_1_ngram_2_squared_ext_model.vw -t -d hw8_data/stackoverflow_test.vw \
    -p hw8_data/models/passes_1_ngram_2_squared_ext_test_answers.tsv

Generating 2-grams for all namespaces.
final_regressor = hw8_data/models/passes_1_ngram_2_squared_ext_model.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using cache_file = cache1.vw
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      320
0.500000 1.000000            2            2.0        4        1      134
0.750000 1.000000            4            4.0        7        1      174
0.750000 0.750000            8            8.0        7        1      188
0.750000 0.750000           16           16.0        7        7      416
0.781250 0.812500           32           32.0        7        2      346
0.765625 0.750000           64           64.0        3        3      406
0.664062 0.562500          128          128.0        1        7       56
0

In [59]:
test_true_answers = pd.read_csv('hw8_data/stackoverflow_test_labels.txt')
model_answers = pd.read_csv('hw8_data/models/{}_test_answers.tsv'.format('passes_1_ngram_2_squared_ext'))
accuracy_score(y_true=test_true_answers, y_pred=model_answers)

0.91439607331972217

In [60]:
0.91439607331972217 - 0.910434396866

0.0039616764537221405

### 5. Заключение

В этом задании мы только познакомились с Vowpal Wabbit. Что еще можно попробовать:
 - Классификация с несколькими ответами (multilabel classification, аргумент  `multilabel_oaa`) – формат данных тут как раз под такую задачу
 - Настройка параметров VW с hyperopt, авторы библиотеки утверждают, что качество должно сильно зависеть от параметров изменения шага градиентного спуска (`initial_t` и `power_t`). Также можно потестировать разные функции потерь – обучать логистическую регресиию или линейный SVM
 - Познакомиться с факторизационными машинами и их реализацией в VW (аргумент `lrq`)